## Mikan-tutorial

欢迎使用 mikan 🍊! 

mikan 是一个医学图像分割指标计算工具。此文档介绍了 mikan 的使用方法，简单对比了其与 medpy 的性能和结果。

mikan 的主要特点：
- rust 驱动，高度并行化，非常快，比现有工具 (medpy 等) 快 10 ~ 100x
- 几乎支持全部的分割指标的计算，和 medpy 结果完全一致
- 精心设计的简单、灵活、符合直觉的接口，你一分钟就可以精通 mikan 并喜欢上它！

### 太长不看

立刻运行 mikan 获得全部分割标签的全部指标：

In [1]:
from mikan import all

all_metrics = all(rf"..\data\patients_26_ground_truth.nii.gz", rf"..\data\patients_26_segmentation.nii.gz")
print(all_metrics)

[{'accuracy': 0.99887297469187, 'adjust_rand_score': 0.9864214165654426, 'amcc': 0.9864367567296637, 'assd': 0.4627411239979557, 'auc': 0.9907127577891529, 'balanced_accuracy': 0.9907127577891529, 'dice': 0.9870104666571462, 'f_score': 0.9870104666571462, 'fn': 136914.0, 'fnr': 0.018227270960283256, 'fp': 57192.0, 'fpr': 0.00034721346141101695, 'hausdorff_distance': 23.23409712643984, 'hausdorff_distance_95': 1.161103780937166, 'jaccard_score': 0.9743540620800129, 'kappa': 0.9864214165654426, 'label': 1.0, 'masd': 0.4570258359112762, 'mcc': 0.9864367567296637, 'nmcc': 0.9932183783648318, 'precision': 0.9923043904749475, 'senstivity': 0.9817727290397168, 'specificity': 0.999652786538589, 'tn': 164659924.0, 'tp': 7374578.0, 'volume_similarity': 0.9946650202613058}, {'accuracy': 0.9994664649440818, 'adjust_rand_score': 0.9635612411025513, 'amcc': 0.963574881429292, 'assd': 0.6030765488980315, 'auc': 0.9843198140671667, 'balanced_accuracy': 0.9843198140671667, 'dice': 0.9638299832080431, '

### 使用

In [2]:
import mikan
import SimpleITK as sitk
import time
from rich import print

mikan 目前只支持 uint8 的数据，通常这是足够的

In [3]:
gt = sitk.ReadImage(rf"..\data\patients_26_ground_truth.nii.gz", sitk.sitkUInt8)
pred = sitk.ReadImage(rf"..\data\patients_26_segmentation.nii.gz", sitk.sitkUInt8)

使用只需要两步：

1. 初始化一个评估器：

In [4]:
e = mikan.Evaluator(gt, pred)

2. 获得指标：

In [5]:
e.labels(1).metrics("dice") # label 1 的 Dice

0.9870104666571462

In [6]:
e.labels([1,2,3]).metrics("Dice") # 多个 label 的 Dice

[0.9870104666571462, 0.9638299832080431, 0.9788861062581474]

In [7]:
e.labels(1).metrics(["Dice", "TP"]) # label 1 的多个 metrics

[0.9870104666571462, 7374578.0]

In [8]:
e.labels([1, 2]).metrics(["Dice", "TP"]) # 多个 label 的多个 metrics

{'1': {'Dice': 0.9870104666571462, 'TP': 7374578.0},
 '2': {'Dice': 0.9638299832080431, 'TP': 1224306.0}}

In [9]:
e.labels("all").metrics(["Dice", "TP"]) # 全部 label 的多个 metrics

{'1': {'Dice': 0.9870104666571462, 'TP': 7374578.0},
 '2': {'Dice': 0.9638299832080431, 'TP': 1224306.0},
 '3': {'Dice': 0.9788861062581474, 'TP': 4379325.0},
 '4': {'Dice': 0.9847150871159485, 'TP': 6222520.0},
 '5': {'Dice': 0.9799666340747192, 'TP': 4541233.0}}

In [10]:
e.labels("all").metrics("all") # 全部 label 的全部 metrics

{'1': {'mcc': 0.9864367567296637,
  'assd': 0.4627411239979557,
  'accuracy': 0.99887297469187,
  'tn': 164659924.0,
  'senstivity': 0.9817727290397168,
  'volume_similarity': 0.9946650202613058,
  'fnr': 0.018227270960283256,
  'fn': 136914.0,
  'adjust_rand_score': 0.9864214165654426,
  'fpr': 0.00034721346141101695,
  'dice': 0.9870104666571462,
  'hausdorff_distance': 23.23409712643984,
  'auc': 0.9907127577891529,
  'hausdorff_distance_95': 1.161103780937166,
  'nmcc': 0.9932183783648318,
  'tp': 7374578.0,
  'specificity': 0.999652786538589,
  'masd': 0.4570258359112762,
  'f_score': 0.9870104666571462,
  'precision': 0.9923043904749475,
  'fp': 57192.0,
  'jaccard_score': 0.9743540620800129,
  'balanced_accuracy': 0.9907127577891529,
  'amcc': 0.9864367567296637,
  'kappa': 0.9864214165654426},
 '2': {'mcc': 0.963574881429292,
  'assd': 0.6030765488980315,
  'accuracy': 0.9994664649440818,
  'tn': 170912412.0,
  'senstivity': 0.9689476091811748,
  'volume_similarity': 0.99471836

### 高效


尽管计算 Dice 很简单，mikan 也进行了高度优化，使得计算 Dice 特别快，比如：

In [11]:
from medpy.metric import dc

e = mikan.Evaluator(gt, pred)

# mikan: DSC
t = time.time()
mikan_dscs = e.labels([1,2,3,4,5]).metrics("dsc")
mikan_costs = time.time() - t

# medpy: DSC
t = time.time()

pred_arr = sitk.GetArrayFromImage(pred)
gt_arr = sitk.GetArrayFromImage(gt)

dscs = []
for i in (1,2,3,4,5):
    dscs.append(dc(pred_arr == i, gt_arr == i))
medpy_costs = time.time() - t

print(f"mikan DSC: {medpy_costs / mikan_costs :.2f}x faster")
print(f"mikan DSC: {mikan_dscs}")
print(f"medpy DSC: {dscs}")

mikan DSC: 8.03x faster

mikan DSC: [0.9870104666571462, 0.9638299832080431, 0.9788861062581474, 0.9847150871159485, 0.9799666340747192]

medpy DSC: [0.9870104666571462, 0.9638299832080431, 0.9788861062581474, 0.9847150871159485, 0.9799666340747192]

mikan 有超快的 Hausdorff 距离计算：

In [12]:
t = time.time()
mikan_hd = e.labels(1).metrics("hd")
print(f"Mikan cost {time.time() - t:.2f} s, HD = {mikan_hd:.4f} mm")

Mikan cost 4.35 s, HD = 23.2341 mm

对比 medpy：

In [13]:
from medpy.metric import hd
t = time.time()
medpy_hd = hd(pred_arr == 1, gt_arr == 1, voxelspacing=gt.GetSpacing()[::-1]) # z, y, x
print(f"medpy cost {time.time() - t:.4f} s, HD = {medpy_hd:.4f} mm")

medpy cost 130.6421 s, HD = 23.2341 mm

### 超级缓存

在计算 HD 时，ASSD、HD95 等指标也被 mikan 计算过了。不需要再计算！

In [14]:
t = time.time()
mikan_hd = e.labels(1).metrics("assd")
print(f"Mikan cost {time.time() - t:.4f} s, ASSD = {mikan_hd:.4f} mm")

Mikan cost 0.0000 s, ASSD = 0.4627 mm

而 medpy 计算 assd 要从头再来，非常慢：

In [15]:
from medpy.metric import assd
t = time.time()
medpy_hd = assd(pred_arr == 1, gt_arr == 1, voxelspacing=gt.GetSpacing()[::-1]) # z, y, x
print(f"medpy cost {time.time() - t:.2f} s, ASSD = {medpy_hd:.4f} mm")

medpy cost 127.19 s, ASSD = 0.4627 mm

再举一个刚刚计算过的 Dice 的例子：

In [16]:
from medpy.metric import dc

# mikan: DSC
t = time.time()
mikan_dscs = e.labels([1,2,3,4,5]).metrics("dsc")
mikan_costs = time.time() - t + 1e-10 # sometimes zero

# medpy: DSC
t = time.time()
dscs = []
for i in (1,2,3,4,5):
    dscs.append(dc(pred_arr == i, gt_arr == i))
medpy_costs = time.time() - t

print(f"mikan DSC: {medpy_costs / mikan_costs :.2f}x faster")

mikan DSC: 18841872215.27x faster

mikan 不会重复计算已经计算过的数据。这就是 mikan 的超级缓存！

### 别名

mikan 也支持别名。你可以把 "Dice" 称为 "dsc"： 

In [17]:
e.labels(1).metrics("dsc") # dsc 也是 Dice！

0.9870104666571462

我们维护了一个别名字典，你可以任意使用左边的名字，他们都会被视为右边进行计算。

In [18]:
from mikan.alias import ALIAS_DICT
print(ALIAS_DICT)

{
    'accuracy': 'accuracy',
    'acc': 'accuracy',
    'Acc': 'accuracy',
    'ACC': 'accuracy',
    'Accuracy': 'accuracy',
    'RI': 'accuracy',
    'RandIndex': 'accuracy',
    'adjust_rand_score': 'adjust_rand_score',
    'auc': 'auc',
    'AUC': 'auc',
    'AUC_trapezoid': 'auc',
    'balanced_accuracy': 'balanced_accuracy',
    'bacc': 'balanced_accuracy',
    'BACC': 'balanced_accuracy',
    'BalancedAccuracy': 'balanced_accuracy',
    'dice': 'dice',
    'Dice': 'dice',
    'dsc': 'dice',
    'DSC': 'dice',
    'f_score': 'f_score',
    'jaccard_score': 'jaccard_score',
    'iou': 'jaccard_score',
    'IoU': 'jaccard_score',
    'jaccard': 'jaccard_score',
    'Jaccard': 'jaccard_score',
    'IntersectionOverUnion': 'jaccard_score',
    'kappa': 'kappa',
    'KAP': 'kappa',
    'Kappa': 'kappa',
    'CohensKappa': 'kappa',
    'mcc': 'mcc',
    'MCC': 'mcc',
    'MatthewsCorrelationCoefficient': 'mcc',
    'nmcc': 'nmcc',
    'NMCC': 'nmcc',
    'nMCC': 'nmcc',
    'MCC_normalized': 'nmcc',
    'amcc': 'amcc',
    'AMCC': 'amcc',
    'aMCC': 'amcc',
    'MCC_absolute': 'amcc',
    'precision': 'precision',
    'prec': 'precision',
    'PREC': 'precision',
    'Precision': 'precision',
    'senstivity': 'senstivity',
    'sens': 'senstivity',
    'SENS': 'senstivity',
    'Sensitivity': 'senstivity',
    'Recall': 'senstivity',
    'recall': 'senstivity',
    'TPR': 'senstivity',
    'tpr': 'senstivity',
    'true_positive_rate': 'senstivity',
    'specificity': 'specificity',
    'Specificity': 'specificity',
    'spec': 'specificity',
    'Spec': 'specificity',
    'SPEC': 'specificity',
    'TNR': 'specificity',
    'tnr': 'specificity',
    'true_negative_rate': 'specificity',
    'tn': 'tn',
    'TN': 'tn',
    'TrueNegative': 'tn',
    'tp': 'tp',
    'TP': 'tp',
    'TruePositive': 'tp',
    'fn': 'fn',
    'FN': 'fn',
    'FalseNegative': 'fn',
    'fp': 'fp',
    'FP': 'fp',
    'FalsePositive': 'fp',
    'fnr': 'fnr',
    'FNR': 'fnr',
    'fpr': 'fpr',
    'FPR': 'fpr',
    'volume_similarity': 'volume_similarity',
    'VolumetricSimilarity': 'volume_similarity',
    'vs': 'volume_similarity',
    'VS': 'volume_similarity',
    'hausdorff_distance_95': 'hausdorff_distance_95',
    'HD95': 'hausdorff_distance_95',
    'hd95': 'hausdorff_distance_95',
    'hausdorff_distance': 'hausdorff_distance',
    'hd': 'hausdorff_distance',
    'HD': 'hausdorff_distance',
    'assd': 'assd',
    'ASSD': 'assd',
    'masd': 'masd',
    'MASD': 'masd'
}

### 结果一致性

和 medpy 完全一致。放心使用！


### 总结

- mikan 能计算几乎全部的分割指标
- mikan 特别快
- mikan 使用特别简单
